In [14]:
import torch
import torch.nn as nn
import math
from torchsummary import summary

# conv함수 설정

In [15]:
def conv_3x3_bn(in_channel, out_channel, stride):
    return nn.Sequential(
        nn.Conv2d(in_channel, out_channel,kernel_size=3, stride=stride, padding=1, bias=False),
        nn.BatchNorm2d(out_channel),
        nn.ReLU6(inplace=True)
    )

def conv_1x1_bn(in_channel, out_channel):
    return nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=1, padding=0, bias=False),
        nn.BatchNorm2d(out_channel),
        nn.ReLU6(inplace=True)
    )

# 채널을 8로 나눌수 있게끔 만들어주는 함수

In [16]:
#모든 채널의 개수를 8로 나눌 수 있게 만들어주는함수이다.
# 이 알고리즘은 나도 잘 모르겠따 그냥 그렇게 해준다고 이해하고 넘겼다.
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)

    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

# invertedResidual블럭 만드는과정

In [17]:
class InvertedResidual(nn.Module):
    def __init__(self, in_channel, out_channel, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        
        #stride가 반드시 1,2이도록 조건을 걸어둔다. 만약에 이상한게 들어오면 오류 출력
        assert stride in [1,2]
        
        hidden_dim = round(in_channel * expand_ratio)

        self.identity = ((stride == 1) and (in_channel == out_channel))

        #channel 수 변경이 없는 경우 expand_ratio가 1이되면서 channel수의 변경이 없어진다.
        if expand_ratio == 1:
            self.conv = nn.Sequential(
                #depthwise과정이다.
                nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),

                #poinwise-linear과정이다.
                nn.Conv2d(hidden_dim, out_channels=out_channel, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_channel)
            )
        
        #channel수 변경이 생기는 경우 바뀌게 될 채널이 
        else:
            self.conv = nn.Sequential(
                #pointwise
                nn.Conv2d(in_channels=in_channel, out_channels=hidden_dim, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),

                #depthwise
                nn.Conv2d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim,bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),

                #pointwise-linear
                nn.Conv2d(in_channels=hidden_dim, out_channels=out_channel, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_channel),
            )
    
    def forward(self, x):
        # skipconnection으로 진행
        if self.identity:
            return x + self.conv(x)
        # stride가 2가되면서 skipconnection이 힘들어지기때문에 그냥 진행한다.
        else:
            return self.conv(x)



# 모바일넷 구현

In [18]:
class MobileNetV2(nn.Module):
    def __init__(self, num_classes = 1000, width_mult=1):
        super(MobileNetV2, self).__init__()

        self.cfgs = [ 
        # t,c,n,s -> expansion, output cnannel, 반복횟수, stride
            [1, 16,  1, 1],
            [6, 24,  2, 2],
            [6, 32,  3, 2],
            [6, 64,  4, 2],
            [6, 96,  3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        #첫번째 레이어 생성
        #channel생성 : 4/8로 나눠 떨어지게
        input_channel = _make_divisible(32*width_mult, 4 if width_mult == 0.1 else 8)
        layers = [conv_3x3_bn(in_channel=3, out_channel=input_channel, stride=2)]

        #inverted residual block만들기
        block = InvertedResidual
        for t, c, n, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 4 if width_mult == 0.1 else 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i==0 else 1, t))
                input_channel = output_channel
        self.feature = nn.Sequential(*layers) #지금까지만든거 feature에다가 모으는거

        #마지막 레이어 생성
        output_channel = _make_divisible(1280 * width_mult, 4 if width_mult == 0.1 else 8) if width_mult > 1.0 else 1280
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()
    
    def forward(self, x):
        x = self.feature(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    #초기화 함수
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
    

In [22]:
model = MobileNetV2()
pred = model(torch.randn(1,3,224,224))
print(pred.shape)

torch.Size([1, 1000])
